# Importing libraries

In [1]:
from police_api import PoliceAPI
import pandas as pd
import re
import numpy as np

# Defining main variables

We assign all england forces to a variable and define a london force by selecting london force from all england forces.

In [2]:
api = PoliceAPI()

In [3]:
all_forces = api.get_forces()    

In [4]:
all_forces[24]

<Force> Metropolitan Police Service

In [5]:
london_force = all_forces[24] 

In [6]:
london_nei = london_force.neighbourhoods     #assigning neighbourhoods in london in variable

# Creating neighbourhood code and name dataframe


In [7]:
df_code = pd.DataFrame({'code': london_nei})       #creating dataframe with neighbourhood code
df_code.head()

,code
0,<Neighbourhood> E05000026
1,<Neighbourhood> E05000455
2,<Neighbourhood> E05000630
3,<Neighbourhood> E05000214
4,<Neighbourhood> E05009388


In [8]:
df_code['neighbourhood'] = df_code.apply(lambda x: x['code'].name, axis=1)

#assigning neighbourhood names to the neighbourhood code and adding a new column with them to the dataframe

df_code.head()

,code,neighbourhood
0,<Neighbourhood> E05000026,Abbey
1,<Neighbourhood> E05000455,Abbey
2,<Neighbourhood> E05000630,Abbey Road
3,<Neighbourhood> E05000214,Abbey Wood
4,<Neighbourhood> E05009388,Abingdon


In [9]:
df_code.code=df_code.code.astype(str)  
df_code['code']=df_code['code'].apply(lambda x:x.split(">")[1])
df_code #cleaning NH column, leaving the digit code

,code,neighbourhood
0,E05000026,Abbey
1,E05000455,Abbey
2,E05000630,Abbey Road
3,E05000214,Abbey Wood
4,E05009388,Abingdon
...,...,...
653,E05000230,Woolwich Riverside
654,E05000572,Worcester Park
655,E05000265,Wormholt and White City
656,E05000344,Yeading


# Main loop to import data from api

In [10]:
#crimes=[]
#nei_crimes=[]
#officers=[]
#nei_officers=[]
#longitudes=[]
#latitudes=[]
#for i in london_nei:
#    for z in i.officers:
#        nei_officers.append(i.name)
#        officers.append(z)   
#    for j in i.crimes:       
#        crimes.append(i.name)
#        crimes.append(j.category)
#        longitudes.append(j.location.longitude)
#        latitudes.append(j.location.latitude)

In [11]:
pd.DataFrame({'neighborhood': [i for i in london_nei] })

,neighborhood
0,<Neighbourhood> E05000026
1,<Neighbourhood> E05000455
2,<Neighbourhood> E05000630
3,<Neighbourhood> E05000214
4,<Neighbourhood> E05009388
...,...
653,<Neighbourhood> E05000230
654,<Neighbourhood> E05000572
655,<Neighbourhood> E05000265
656,<Neighbourhood> E05000344


In [19]:
df_crimes.reset_index(inplace=True)
df_officers.reset_index(inplace=True)

In [13]:
df_crimes = pd.DataFrame()
i = 0
for n in london_nei:
    df_temp = pd.DataFrame({'crimes': [c.category.name for c in n.crimes],
                           'longitude': [c.location.longitude for c in n.crimes],
                            'latitude': [c.location.latitude for c in n.crimes]
                           })
    df_temp['neighbourhood'] = n.name
    df_crimes = df_crimes.append(df_temp)
    i += 1
    print('Processed neighbourhood number  ',i)
#     if i > 4:
#         break
# df_crimes        
df_crimes.to_csv('crimesneighbourhood3.csv')

Processed neighbourhood number   1
Processed neighbourhood number   2
Processed neighbourhood number   3
Processed neighbourhood number   4
Processed neighbourhood number   5
Processed neighbourhood number   6
Processed neighbourhood number   7
Processed neighbourhood number   8
Processed neighbourhood number   9
Processed neighbourhood number   10
Processed neighbourhood number   11
Processed neighbourhood number   12
Processed neighbourhood number   13
Processed neighbourhood number   14
Processed neighbourhood number   15
Processed neighbourhood number   16
Processed neighbourhood number   17
Processed neighbourhood number   18
Processed neighbourhood number   19
Processed neighbourhood number   20
Processed neighbourhood number   21
Processed neighbourhood number   22
Processed neighbourhood number   23
Processed neighbourhood number   24
Processed neighbourhood number   25
Processed neighbourhood number   26
Processed neighbourhood number   27
Processed neighbourhood number   28
P

Processed neighbourhood number   226
Processed neighbourhood number   227
Processed neighbourhood number   228
Processed neighbourhood number   229
Processed neighbourhood number   230
Processed neighbourhood number   231
Processed neighbourhood number   232
Processed neighbourhood number   233
Processed neighbourhood number   234
Processed neighbourhood number   235
Processed neighbourhood number   236
Processed neighbourhood number   237
Processed neighbourhood number   238
Processed neighbourhood number   239
Processed neighbourhood number   240
Processed neighbourhood number   241
Processed neighbourhood number   242
Processed neighbourhood number   243
Processed neighbourhood number   244
Processed neighbourhood number   245
Processed neighbourhood number   246
Processed neighbourhood number   247
Processed neighbourhood number   248
Processed neighbourhood number   249
Processed neighbourhood number   250
Processed neighbourhood number   251
Processed neighbourhood number   252
P

APIError: <unknown error code>

In [14]:
df_officers = pd.DataFrame()
i = 0
for n in london_nei:
    df_temp = pd.DataFrame({'officers': [o.name for o in n.officers]})
    df_temp['neighborhoud'] = n.name
    df_officers = df_officers.append(df_temp)
    i += 1
    print('Processed neighbourhood number  ',i)

df_officers.to_csv('officersneighborhoods3.csv')

Processed neighbourhood number   1
Processed neighbourhood number   2
Processed neighbourhood number   3
Processed neighbourhood number   4
Processed neighbourhood number   5
Processed neighbourhood number   6
Processed neighbourhood number   7
Processed neighbourhood number   8
Processed neighbourhood number   9
Processed neighbourhood number   10
Processed neighbourhood number   11
Processed neighbourhood number   12
Processed neighbourhood number   13
Processed neighbourhood number   14
Processed neighbourhood number   15
Processed neighbourhood number   16
Processed neighbourhood number   17
Processed neighbourhood number   18
Processed neighbourhood number   19
Processed neighbourhood number   20
Processed neighbourhood number   21
Processed neighbourhood number   22
Processed neighbourhood number   23
Processed neighbourhood number   24
Processed neighbourhood number   25
Processed neighbourhood number   26
Processed neighbourhood number   27
Processed neighbourhood number   28
P

Processed neighbourhood number   226
Processed neighbourhood number   227
Processed neighbourhood number   228
Processed neighbourhood number   229
Processed neighbourhood number   230
Processed neighbourhood number   231
Processed neighbourhood number   232
Processed neighbourhood number   233
Processed neighbourhood number   234
Processed neighbourhood number   235
Processed neighbourhood number   236
Processed neighbourhood number   237
Processed neighbourhood number   238
Processed neighbourhood number   239
Processed neighbourhood number   240
Processed neighbourhood number   241
Processed neighbourhood number   242
Processed neighbourhood number   243
Processed neighbourhood number   244
Processed neighbourhood number   245
Processed neighbourhood number   246
Processed neighbourhood number   247
Processed neighbourhood number   248
Processed neighbourhood number   249
Processed neighbourhood number   250
Processed neighbourhood number   251
Processed neighbourhood number   252
P

Processed neighbourhood number   449
Processed neighbourhood number   450
Processed neighbourhood number   451
Processed neighbourhood number   452
Processed neighbourhood number   453
Processed neighbourhood number   454
Processed neighbourhood number   455
Processed neighbourhood number   456
Processed neighbourhood number   457
Processed neighbourhood number   458
Processed neighbourhood number   459
Processed neighbourhood number   460
Processed neighbourhood number   461
Processed neighbourhood number   462
Processed neighbourhood number   463
Processed neighbourhood number   464
Processed neighbourhood number   465
Processed neighbourhood number   466
Processed neighbourhood number   467
Processed neighbourhood number   468
Processed neighbourhood number   469
Processed neighbourhood number   470
Processed neighbourhood number   471
Processed neighbourhood number   472
Processed neighbourhood number   473
Processed neighbourhood number   474
Processed neighbourhood number   475
P

KeyboardInterrupt: 

In [16]:
df_officers=pd.read_csv('officersneighborhoods3.csv')
df_crimes=pd.read_csv('crimesneighbourhood3.csv')

In [48]:
df_officers=df_officers[['neighbourhood','officers']]
df_crimes=df_crimes[['crimes','longitude','latitude','neighbourhood']]

# Creating dataframe with crimes and their location 

In [49]:
#df_crimes=pd.DataFrame(nei_crimes)  
#df_crimes['crimes']=crimes
#df_crimes['latitude']=latitudes
#df_crimes['longitude']=longitudes
#df_crimes

# New dataframe with neighbourhoods and their officers

In [50]:
#df_officers=pd.DataFrame(nei_officers)  
#df_officers['officers']=officers

In [51]:
#df_officers.officers=df_officers.officers.astype(str)  #cleaning officers column name
#df_officers['officers']=df_officers['officers'].apply(lambda x:x.split(">")[1])
#df_officers.rename(columns={0:'Neighbourhood'},inplace=True)

In [52]:
#df_crimes.crimes=df_crimes.crimes.astype(str)  #cleaning in crimes dataframe
#df_crimes['crimes']=df_crimes['crimes'].apply(lambda x:x.split(">")[1])
#df_crimes

# Removing anti-social behaviour from crime dataframe 

Importing crime dataframe with location to a csv. in order to create a map with tableau.

In [53]:
not_antisocial=df_crimes.loc[df_crimes['crimes']!='Anti-social behaviour']
not_antisocial
not_antisocial.to_csv('/Users/islaam/Desktop/anti.csv')

In [54]:
not_antisocial=not_antisocial.groupby(['crimes']).count()

not_antisocial.sort_values('crimes',ascending=False,inplace=True)

not_antisocial



,longitude,latitude,neighbourhood
crimes,,,
Violence and sexual offences,18499,18499,18499
Vehicle crime,8974,8974,8974
Theft from the person,3197,3197,3197
Shoplifting,3314,3314,3314
Robbery,3128,3128,3128
Public order,4047,4047,4047
Possession of weapons,493,493,493
Other theft,8166,8166,8166
Other crime,770,770,770


# Renaming neighbourhood column in dataframes

In [55]:
#df_officers.rename(columns={0:'Neighbourhood'},inplace=True) 

In [56]:
#df_crimes.rename(columns={0:'Neighbourhood'},inplace=True)

# Grouping crimes by neighbourhood 

In [57]:
crime_type_nh = df_code.merge(df_crimes, how='right').groupby(['neighbourhood','crimes']).count()
#crime_type_nh.drop(columns=['latitude','longitude'],inplace=True) 
crime_type_nh.rename(columns={'code':'total crimes'},inplace=True)
crime_type_nh

total crimes  longitude  latitude
neighbourhood crimes                                                         
Abbey         Anti-social behaviour                  154        154       154
              Bicycle theft                            8          8         8
              Burglary                                28         28        28
              Criminal damage and arson               36         36        36
              Drugs                                   40         40        40
...                                                  ...        ...       ...
Yiewsley      Robbery                                  5          5         5
              Shoplifting                             12         12        12
              Theft from the person                    2          2         2
              Vehicle crime                           16         16        16
              Violence and sexual offences            33         33        33

[7626 rows x 3 columns]

In [58]:
crime_type_nh.sort_values('total crimes',ascending=False,inplace=True)
crime_type_nh

total crimes  longitude  latitude
neighbourhood crimes                                                         
St Peter's    Anti-social behaviour                  330        330       330
Fairfield     Anti-social behaviour                  272        272       272
Regent's Park Anti-social behaviour                  242        242       242
Fairfield     Violence and sexual offences           238        238       238
Soho          Other theft                            235        235       235
...                                                  ...        ...       ...
Queen's Park  Possession of weapons                    1          1         1
              Other crime                              1          1         1
Queen's Gate  Possession of weapons                    1          1         1
              Drugs                                    1          1         1
Clementswood  Other crime                              1          1         1

[7626 rows x 3 columns]

In [59]:
crime_nh=df_crimes.groupby(['neighbourhood']).count()
#crime_nh.drop(columns=['latitude','longitude'],inplace=True) 
crime_nh = crime_nh.sort_values('crimes',ascending=False)
crime_nh.reset_index(inplace=True)

# Grouping officers per neighbourhood

In [69]:
officers_nh = df_officers.groupby(['neighbourhood']).count()
officers_nh = officers_nh.sort_values('officers',ascending=False)
officers_nh.reset_index(inplace=True)

In [70]:
df_officers.rename(columns = {'neighborhoud':'neighbourhood'}, inplace = True)

In [71]:
officers_nh.rename(columns = {'neighborhoud':'neighbourhood'}, inplace = True)

In [72]:
officers_nh.corr()

,officers
officers,1.0


# Merging total crimes per neighbourhood with officers

In [73]:
crimes_officers = crime_nh.merge(officers_nh, how='outer').groupby(['neighbourhood']).sum()
crimes_officers

,crimes,longitude,latitude,officers
neighbourhood,,,,
Abbey,351.0,351.0,351.0,15
Abbey Road,72.0,72.0,72.0,5
Abbey Wood,185.0,185.0,185.0,4
Abingdon,108.0,108.0,108.0,5
Acton Central,141.0,141.0,141.0,5
...,...,...,...,...
Woolwich Riverside,342.0,342.0,342.0,6
Worcester Park,54.0,54.0,54.0,4
Wormholt and White City,102.0,102.0,102.0,3


In [74]:
crimes_officers.sort_values('crimes',ascending=False,inplace=True)

# Counting total crimes in London per category

In [75]:
crimes_london=df_crimes.groupby(['crimes']).count()
crimes_london.drop(columns=['latitude','longitude'],inplace=True) 
crimes_london.rename(columns={'neighbourhood':'total crimes'},inplace=True)
crimes_london.sort_values('total crimes',ascending=False,inplace=True)
crimes_london


,total crimes
crimes,
Anti-social behaviour,19866
Violence and sexual offences,18499
Vehicle crime,8974
Other theft,8166
Burglary,6198
Public order,4047
Criminal damage and arson,3975
Drugs,3646
Shoplifting,3314


In [81]:
crimes_officerss=crimes_officers[['crimes','officers']]

In [83]:
crimes_officerss.corr()

,crimes,officers
crimes,1.000000,0.405115
officers,0.405115,1.000000


In [84]:
crimes_officers.corr()

,crimes,longitude,latitude,officers
crimes,1.000000,1.000000,1.000000,0.405115
longitude,1.000000,1.000000,1.000000,0.405115
latitude,1.000000,1.000000,1.000000,0.405115
officers,0.405115,0.405115,0.405115,1.000000
